# Modelo de aplicación basado en bert para clasificación de textos

Dado el sesgo contextual evidenciado al momento de aplicar una primera categorización de las reseñas con la ayuda de robertuito, se procede con la recolección de clasificación de datos por aprte del equipo humano de la Universidad de Cundinamarca, con lo cual se establece una fuente de datos acorde a una muestra de registros con un intervalo de confianza del 95%, conholgura del 5% y la aplicación de clasificación a ciegas entre los participantes

In [1]:
import pandas as pd
import numpy as np

In [2]:
data_contextual = pd.read_excel("Data_contextual.xlsx")

In [3]:
data_contextual

,Reseña,Pregunta,Categoria,Categoria con contexto
0,nada,¿En cuales aspectos considera que debería mejo...,NEU,POS
1,buena profesora,¿Qué aspectos positivos destaca el profesor?,POS,POS
2,ninguno,¿En cuales aspectos considera que debería mejo...,NEU,POS
3,ninguno,¿Qué aspectos positivos destaca el profesor?,NEU,NEG
4,en nada,¿En cuales aspectos considera que debería mejo...,NEU,POS
...,...,...,...,...
375,una docente que estaca por su orden y cumplimi...,¿Qué aspectos positivos destaca el profesor?,POS,POS
376,ninguno,¿En cuales aspectos considera que debería mejo...,NEU,POS
377,no,¿En cuales aspectos considera que debería mejo...,NEU,POS
378,orden _x000D_ dominio de tema _x000D_ manejo d...,¿En cuales aspectos considera que debería mejo...,POS,NEG


## Encoding de los labels (etiquetas de categorización a reseña)

In [4]:
label_map = {
    "NEG":int(0),
    "NEU":int(1),
    "POS":int(2)
}

In [5]:
data_contextual["labels"] = data_contextual["Categoria con contexto"].map(label_map)
num_labels = len(label_map)

In [6]:
data_contextual

,Reseña,Pregunta,Categoria,Categoria con contexto,labels
0,nada,¿En cuales aspectos considera que debería mejo...,NEU,POS,2.0
1,buena profesora,¿Qué aspectos positivos destaca el profesor?,POS,POS,2.0
2,ninguno,¿En cuales aspectos considera que debería mejo...,NEU,POS,2.0
3,ninguno,¿Qué aspectos positivos destaca el profesor?,NEU,NEG,0.0
4,en nada,¿En cuales aspectos considera que debería mejo...,NEU,POS,2.0
...,...,...,...,...,...
375,una docente que estaca por su orden y cumplimi...,¿Qué aspectos positivos destaca el profesor?,POS,POS,2.0
376,ninguno,¿En cuales aspectos considera que debería mejo...,NEU,POS,2.0
377,no,¿En cuales aspectos considera que debería mejo...,NEU,POS,2.0
378,orden _x000D_ dominio de tema _x000D_ manejo d...,¿En cuales aspectos considera que debería mejo...,POS,NEG,0.0


## Conversion del DataFrame a un Dataset de HuggingFaces

In [7]:
from datasets import Dataset, DatasetDict

d:\Pruebas IA\TFM\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
dataset_general = Dataset.from_pandas(data_contextual)

In [9]:
print("primeras 2 filas del dataset")
print(dataset_general.select(range(3)))
print("Columnas del dataset")
print(dataset_general.column_names)

primeras 2 filas del dataset
Dataset({
    features: ['Reseña', 'Pregunta', 'Categoria', 'Categoria con contexto', 'labels'],
    num_rows: 3
})
Columnas del dataset
['Reseña', 'Pregunta', 'Categoria', 'Categoria con contexto', 'labels']


## Generación de train y test

In [10]:
data_contextual['labels'] = data_contextual['labels'].fillna(0).astype(int)

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
train_df, eval_df = train_test_split(data_contextual, test_size=0.3, random_state=42, stratify=data_contextual['labels'])

In [13]:
dataset_train = Dataset.from_pandas(train_df)
dataset_eval = Dataset.from_pandas(eval_df)

## Carga del tokenizador

In [14]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

In [15]:
model_name = 'dccuchile/bert-base-spanish-wwm-cased'
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [16]:
MAX_LEN = 128

## Crear función de tokenización

In [17]:
def tokenizar(ejemplos):
    return tokenizer(ejemplos['Pregunta'], ejemplos['Reseña'], max_length=MAX_LEN, truncation=True, padding='max_length', return_tensors="pt"
)

In [18]:
dataset_train_tokenizado = dataset_train.map(tokenizar, batched=True)

Map: 100%|██████████| 266/266 [00:00<00:00, 6232.04 examples/s]


In [19]:
dataset_eval_tokenizado = dataset_eval.map(tokenizar, batched=True)

Map: 100%|██████████| 114/114 [00:00<00:00, 7037.32 examples/s]


In [20]:
dataset_train_tokenizado.column_names

['Reseña',
 'Pregunta',
 'Categoria',
 'Categoria con contexto',
 'labels',
 '__index_level_0__',
 'input_ids',
 'token_type_ids',
 'attention_mask']

In [21]:
dataset_train_tokenizado = dataset_train_tokenizado.remove_columns(["Reseña", "Pregunta", "Categoria", "Categoria con contexto", "__index_level_0__"])

In [22]:
dataset_eval_tokenizado = dataset_eval_tokenizado.remove_columns(["Reseña", "Pregunta", "Categoria", "Categoria con contexto", "__index_level_0__"])

In [23]:
print(dataset_train_tokenizado[0])

{'labels': 2, 'input_ids': [4, 1067, 1281, 4909, 12848, 10446, 1040, 5533, 1064, 5, 1079, 2233, 1008, 8509, 1058, 2667, 2803, 14987, 30934, 1062, 21046, 1047, 1477, 2415, 7122, 1058, 1108, 1456, 2667, 15423, 1486, 1008, 22945, 1912, 3894, 2125, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

## Carga del modelo 

In [24]:
modelo = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels = num_labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Configuración de los Training Arguments

In [25]:
training_arguments = TrainingArguments(
    output_dir="./resultados_datasets",
    num_train_epochs= 25,
    per_device_train_batch_size= 8,
    per_device_eval_batch_size= 8,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1"
)

## Definir metricas

In [26]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [27]:
def metricas(p):
    predicciones = np.argmax(p.predictions, axis= 1)

    labels = p.label_ids

    precision, recall, f1, _ = precision_recall_fscore_support(labels, predicciones, average='weighted')
    accuracy = accuracy_score(labels, predicciones)

    return {
        'accuracy': accuracy,
        'f1': f1,
        'recall': recall,
        'precision': precision
    }



## Inicializar el trainer

In [28]:
trainer = Trainer(
    model=modelo,
    args=training_arguments,
    train_dataset= dataset_train_tokenizado,
    eval_dataset= dataset_eval_tokenizado,
    tokenizer = tokenizer,
    compute_metrics=metricas
)

C:\Users\ASANGEL\AppData\Local\Temp\ipykernel_6240\4124256841.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


## Entrenar modelo

In [29]:
trainer.train()

d:\Pruebas IA\TFM\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Recall,Precision
1,No log,0.926998,0.614035,0.467201,0.614035,0.377039
2,No log,0.778677,0.640351,0.523635,0.640351,0.574960
3,0.881100,0.633096,0.736842,0.670332,0.736842,0.639889
4,0.881100,0.503354,0.807018,0.794367,0.807018,0.792713
5,0.881100,0.468803,0.850877,0.845776,0.850877,0.846381
6,0.423500,0.648891,0.807018,0.796385,0.807018,0.798389
7,0.423500,0.861009,0.780702,0.792534,0.780702,0.811356
8,0.423500,0.996203,0.736842,0.707587,0.736842,0.717705
9,0.220700,0.776371,0.789474,0.791124,0.789474,0.794301
10,0.220700,0.837239,0.824561,0.815625,0.824561,0.810656


d:\Pruebas IA\TFM\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Pruebas IA\TFM\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
d:\Pruebas IA\TFM\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Pruebas IA\TFM\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no acc

TrainOutput(global_step=850, training_loss=0.2768799321791705, metrics={'train_runtime': 2311.529, 'train_samples_per_second': 2.877, 'train_steps_per_second': 0.368, 'total_flos': 437426056972800.0, 'train_loss': 0.2768799321791705, 'epoch': 25.0})

In [30]:
trainer.evaluate()

d:\Pruebas IA\TFM\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.8884883522987366,
 'eval_accuracy': 0.8596491228070176,
 'eval_f1': 0.8522942502039181,
 'eval_recall': 0.8596491228070176,
 'eval_precision': 0.8519933396230653,
 'eval_runtime': 8.2827,
 'eval_samples_per_second': 13.764,
 'eval_steps_per_second': 1.811,
 'epoch': 25.0}

In [33]:
output_dir_model = './Modelo_Contextual_BERT_Gladys_V1'

In [34]:
trainer.save_model(output_dir_model)